CPace sage notebook for generating test vectors for the CPace cipher suites.

The notebook is organized in a series of several subcells.
1.) X25519 definitions and basic string->integer->fieldElement->string conversions back and forth.
2.) Definitions of the Elligator2 primitive straight-line code from the Elligator paper, checked also against the code from the hash_to_curve team.
3.) Test vector generation for X25519 and Elligator2 as ported to sage from the ANSI-C code from the
    Endress+Hauser crypto library.
4.) Implementation and test vector generation for the CPace-specific constructions for generator
    calculation and ISK determination.
5.) Implementation and test vector generation for the AuCPace-specific constructions for generator
    calculation and SK determination.
    
The cells should best be calculated in sequence so that all definitions are available.
Tested with SageMath 9.0 on ubuntu linux.

In [25]:
# 1.) X25519 definitions and tool definitions
   
def ByteArrayToInteger(k,numBytes=32):
    return sum((k[i] << (8 * i)) for i in range(len(k)))

def IntegerToByteArray(k,numBytes = 32):
    result = bytearray(numBytes);
    for i in range(numBytes):
        result[i] = (k >> (8 * i)) & 0xff;
    return result

def IntegerToLEPrintString(u,numBytes=32):
    u = Integer(u)
    res = ""
    ctr = 0
    while ((u != 0) | (numBytes > 0)):
        byte =  u % 256
        res += ("%02x" % byte)
        u = (u - byte) >> 8
        numBytes = numBytes - 1
        ctr = ctr + 1
    return res

def ByteArrayToCInitializer(k, name, values_per_line = 12):
    values = [b for b in k]
    result = "const uint8_t " + name +"[] = {"
    n = 0
    for x in values:
        if n == 0:
            result += "\n "
        n = (n + 1) % values_per_line;
        
        result += ("0x%02x" %x) +","
    result += "\n};"
    return result

def ByteArrayToLEPrintString(k):
    bytes = [(b) for b in k]
    res = ""
    ctr = 0
    for x in bytes:
        res += ("%02x" %x)
        ctr = ctr + 1
    return res

def tv_output_byte_array(data, test_vector_name = "", line_prefix = "  ", max_len = 70):
    string = ByteArrayToLEPrintString(data)
    print (line_prefix + test_vector_name + ": (length: %i bytes)" % len(data) ,end="")
    chars_per_line = max_len - len(line_prefix)
    while True:
        print ("\n" + line_prefix + "  " + string[0:chars_per_line],end="")
        string = string[chars_per_line:-1]
        if len(string) == 0:
            print("\n",end="")
            return

In [2]:
########## Definitions from RFC 7748 ##################

def decodeLittleEndian(b, bits):
    return sum([b[i] << 8*i for i in range(floor((bits+7)/8))])

def string_or_bytes_to_list(u):
    try:
        u_list = [ord(b) for b in u]
    except:
        u_list = [b for b in u]
    return u_list

def decodeUCoordinate(u, bits):
    u_list = string_or_bytes_to_list(u)    
    # Ignore any unused bits.
    if bits % 8:
        u_list[-1] &= (1<<(bits%8))-1
    return decodeLittleEndian(u_list, bits)

def encodeUCoordinate(u, bits):
    u = Integer(u) % p
    return ''.join([chr((u >> 8*i) & 0xff)
                    for i in range((bits+7)/8)])

def decodeScalar25519(k):
    k_list = string_or_bytes_to_list(k)    
    k_list[0] &= 248
    k_list[31] &= 127
    k_list[31] |= 64
    return decodeLittleEndian(k_list, 255)

def decodeScalar448(k):
    k_list = string_or_bytes_to_list(k)    
    k_list[0] &= 252
    k_list[55] |= 128
    return decodeLittleEndian(k_list, 448)

########## Additions ##################

def decodeScalarForInverse25519(k):
    k_list = string_or_bytes_to_list(k)    
    k_list[0] &= 248
    return decodeLittleEndian(k_list, 255)

def decodeScalarForInverse448(k):
    k_list = string_or_bytes_to_list(k)    
    k_list[0] &= 252
    return decodeLittleEndian(k_list, 448)

def decodeUnclampedScalar(k):
    k_list = string_or_bytes_to_list(k)    
    return decodeLittleEndian(k_list, len(k_list) * 8)

########## X25519 ##################

# all inputs to be given as byte array.
def Inverse_X25519(scalar,basepoint):
    OrderPrimeSubgroup = 2^252 + 27742317777372353535851937790883648493
    SF = GF(OrderPrimeSubgroup)
    coFactor = 8
    scalar_clamped = decodeScalar25519(scalar)
    inverse_scalar = 1 /  (SF(scalarClamped) * coFactor)
    inverse_scalar_int = Integer(inverse_scalar) * coFactor
    inverse_scalar = IntegerToByteArray(inverse_scalar_int)
    return X__(basepoint,inverse_scalar,
               scalar_decoder=decodeScalarForInverse25519,
               warnForPointOnTwist = warnForPointOnTwist,
               A = 486662, field_prime = 2^255-19)

def X25519(scalar, basepoint, warnForPointOnTwist = True):
    return X__(scalar, basepoint, 
               scalar_decoder = decodeScalar25519, 
               warnForPointOnTwist = warnForPointOnTwist, 
               A = 486662, field_prime = 2^255-19)

A_Curve25519 = 486662
q_Curve25519 = 2^255-19

########## X448 ##################

A_Curve448 = 156326
q_Curve448 = 2^448 - 2^224 - 1

# all inputs to be given as byte array.
def Inverse_X448(scalar,basepoint):
    OrderPrimeSubgroup = 2^446 - 0x8335dc163bb124b65129c96fde933d8d723a70aadc873d6d54a7bb0d
    num_bytes_for_field = ceil(log(q_Curve448,2) / 8)

    SF = GF(OrderPrimeSubgroup)
    coFactor = 4
    scalar_clamped = decodeScalar448(scalar)
    inverse_scalar = 1 /  (SF(scalarClamped) * coFactor)
    inverse_scalar_int = Integer(inverse_scalar) * coFactor
    inverse_scalar = IntegerToByteArray(inverse_scalar_int,num_bytes_for_field)
    return X__(basepoint,inverse_scalar,
               scalar_decoder=decodeScalarForInverse448,
               warnForPointOnTwist = warnForPointOnTwist,
               A = 156326, field_prime = 2^448 - 2^224 - 1)

def X448(scalar, basepoint, warnForPointOnTwist = True):
    return X__(scalar, basepoint, 
               scalar_decoder = decodeScalar448, 
               warnForPointOnTwist = warnForPointOnTwist, 
               A = 156326, field_prime = 2^448 - 2^224 - 1)

########## Common for X448 and X25519 ##################

def is_on_curve(basepoint, A = 486662, field_prime = 2^255-19):
    F = GF(field_prime)
    A = F(A)
    u = F(ByteArrayToInteger(basepoint))
    v2 = u^3 + A*u^2 + u
    if not v2.is_square():
        return  False
    else:
        return True # on twist

def get_nonsquare(F):
    """ Argument: F, a field object, e.g., F = GF(2^255 - 19) """
    ctr = F.gen()
    while True:
        for Z_cand in (F(ctr), F(-ctr)):
            # Z must be a non-square in F.
            if is_square(Z_cand):
                continue
            return Z_cand
        ctr += 1

def X__(encoded_scalar, basepoint, scalar_decoder=decodeScalar25519, 
        warnForPointOnTwist = True, 
        A = 486662, field_prime = 2^255-19):
    """Implements scalar multiplication for both, X448 and X25519."""
    num_bytes_for_field = ceil(log(field_prime,2) / 8)
    F = GF(Integer(field_prime))
    A = F(A)
    nonsquare = get_nonsquare(F)
    E = EllipticCurve(F, [0, A , 0, 1 , 0])
    Twist = EllipticCurve(F, [0, A * nonsquare, 0, 1 * nonsquare^2, 0])

    u = F(ByteArrayToInteger(basepoint,num_bytes_for_field))
    scalar = scalar_decoder(encoded_scalar)

    d = 1
    v2 = u^3 + A*u^2 + u
    if not v2.is_square():
        if (warnForPointOnTwist):
            print("Input point is on the twist! "),
        E = Twist
        d = nonsquare
        u = d * u
        v2 = u^3 + A*u^2 * nonsquare + u * nonsquare^2
    v = v2.sqrt()
    
    point = E(u, v)
    (resultPoint_u, resultPoint_v, result_Point_z) = point * scalar
    resultCoordinate = resultPoint_u / d
    
    return IntegerToByteArray(Integer(resultCoordinate),num_bytes_for_field)

In [3]:
def montgomery_get_weak_points(A,field_prime):
    F = GF(field_prime)
    A = F(A)
    num_bytes_for_field = ceil(log(field_prime,2) / 8)

    nonsquare = get_nonsquare(F)   
    curve = EllipticCurve(F, [0, A , 0, 1 , 0])
    twist = EllipticCurve(F, [0, A * nonsquare, 0, 1 * nonsquare^2, 0])
    
    order_curve = curve.order()
    order_twist = twist.order()
       
    def get_cofactor(order_curve):
        cofactor_candidate = 1;
        while True:
            if order_curve % cofactor_candidate == 0:
                rest = Integer(order_curve / cofactor_candidate)
                if rest.is_prime():
                    return Integer(cofactor_candidate), Integer(order_curve / cofactor_candidate);
            cofactor_candidate += 1

    c,p = get_cofactor(order_curve)
    c_prime,p_prime = get_cofactor(order_twist)
    
    print ("Number of bytes for field: %i" % num_bytes_for_field)
    print ("Cofactor curve: %i" % c)
    print ("Cofactor twist: %i" % c_prime)
    print ("Order curve: %i" % (p))
    print ("Order twist: %i" % (p_prime))
    
    weak_points = [];

    for m in range(100):
        u = IntegerToByteArray(m,num_bytes_for_field)

        if is_on_curve(u,A,field_prime):
            u1 = X__(IntegerToByteArray(p,num_bytes_for_field),u,
                     scalar_decoder = decodeUnclampedScalar,
                     warnForPointOnTwist = False)
            if not u1 in weak_points:
                weak_points.append(u1)
        else:
            u1 = X__(IntegerToByteArray(p_prime,num_bytes_for_field),u,
                     scalar_decoder = decodeUnclampedScalar,
                     warnForPointOnTwist = False)
            if not u1 in weak_points:
                weak_points.append(u1)
        if len(weak_points) == (c + c_prime) / 2 - 1: break

    non_canonical_weak_points = []
    for m in weak_points:
        u = ByteArrayToInteger(m)
        while True:
            u += field_prime
            if u < 2^(8 * num_bytes_for_field):
                non_canonical_weak_points.append(IntegerToByteArray(u,num_bytes_for_field))
            else:
                break;

    return weak_points,non_canonical_weak_points

weak_pts, nc_weak_pts = montgomery_get_weak_points(A_Curve25519,q_Curve25519)
print (len(weak_pts))
print (len(nc_weak_pts))

Number of bytes for field: 32
Cofactor curve: 8
Cofactor twist: 4
Order curve: 7237005577332262213973186563042994240857116359379907606001950938285454250989
Order twist: 14474011154664524427946373126085988481603263447650325797860494125407373907997
5
7


In [4]:
input_scalar = 599189175373896402783756016145213256157230856085026129926891459468622403380588640249457727683869421921443004045221642549886377526240828
input_coor =   382239910814107330116229961234899377031416365240571325148346555922438025162094455820962429142971339584360034337310079791515452463053830
output_coor =  0xce3e4ff95a60dc6697da1db1d85e6afbdf79b50a2412d7546d5f239fe14fbaadeb445fc66a01b0779d98223961111e21766282f73dd96b6f

print (X448(IntegerToByteArray(input_scalar,57),IntegerToByteArray(input_coor,57)))

input_scalar = 31029842492115040904895560451863089656472772604678260265531221036453811406496
input_coor =   34426434033919594451155107781188821651316167215306631574996226621102155684838
output_coor = 0xc3da55379de9c6908e94ea4df28d084f32eccf03491c71f754b4075577a28552

X25519(IntegerToByteArray(input_scalar),IntegerToByteArray(input_coor))

bytearray(b'\xce>O\xf9Z`\xdcf\x97\xda\x1d\xb1\xd8^j\xfb\xdfy\xb5\n$\x12\xd7Tm_#\x9f\xe1O\xba\xad\xebD_\xc6j\x01\xb0w\x9d\x98"9a\x11\x1e!vb\x82\xf7=\xd9ko')


bytearray(b'\xc3\xdaU7\x9d\xe9\xc6\x90\x8e\x94\xeaM\xf2\x8d\x08O2\xec\xcf\x03I\x1cq\xf7T\xb4\x07Uw\xa2\x85R')

In [5]:
# 2.) Definitions for the X25519 test cases

class X25519_testCase:
    def __init__(self,u_in, s_in, u_out):
        self.u_in = u_in
        self.s_in = s_in
        self.u_out = u_out

    def runTest(self):
        us = IntegerToByteArray(self.u_in)
        ss = IntegerToByteArray(self.s_in)
        r  = IntegerToByteArray(self.u_out)
        u = X25519(ss,us)
        if (u != r):
            print ("Fail")
            print ("Input u :\n0x%032x\n" % self.u_in)
            print ("Input s :\n0x%032x\n" % self.s_in)
            print ("Correct Result :\n0x%032x\n" % self.u_out)
            print ("Actual Result :\n0x%032x\n" % ByteArrayToInteger(u))
            return False
        print ("Pass")
        return True
    
    def docOutput(self):
        print ("Test case for X25519:")
        print ("u:"),
        print (IntegerToLEPrintString(self.u_in))
        print ("s:"),
        print (IntegerToLEPrintString(self.s_in))
        print ("r:"),
        print (IntegerToLEPrintString(self.u_out))
        

testCases = []

tv = \
    X25519_testCase(0x4c1cabd0a603a9103b35b326ec2466727c5fb124a4c19435db3030586768dbe6,\
                    0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                    0x5285a2775507b454f7711c4903cfec324f088df24dea948e90c6e99d3755dac3)
testCases.append(tv)


tv = X25519_testCase(0x13a415c749d54cfc3e3cc06f10e7db312cae38059d95b7f4d3116878120f21e5,\
                     0xdba18799e16a42cd401eae021641bc1f56a7d959126d25a3c67b4d1d4e9664b,\
                    0x5779ac7a64f7f8e652a19f79685a598bf873b8b45ce4ad7a7d90e87694decb95)
testCases.append(tv)

tv = X25519_testCase(0,\
                     0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                     0)
testCases.append(tv)
    
weakp = []
weakp.append(0)
weakp.append(1)
weakp.append(325606250916557431795983626356110631294008115727848805560023387167927233504) #(which has order 8)
weakp.append(39382357235489614581723060781553021112529911719440698176882885853963445705823) #(which also has order 8)
weakp.append(2^255 - 19 - 1)
weakp.append(2^255 - 19)
weakp.append(2^255 - 19 + 1)
weakp.append(2^255 - 19 + 325606250916557431795983626356110631294008115727848805560023387167927233504)
weakp.append(2^255 - 19 + 39382357235489614581723060781553021112529911719440698176882885853963445705823)
weakp.append(2 * (2^255 - 19) - 1)
weakp.append(2 * (2^255 - 19))
weakp.append(2 * (2^255 - 19) + 1)

s_in = 0xff9a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346af;
for x in weakp:
    tv = X25519_testCase (x,s_in,0)
    testCases.append(tv)


for x in testCases:
    x.runTest()

for x in testCases:
    x.docOutput()


Pass
Input point is on the twist! 
Pass
Pass
Pass
Pass
Pass
Pass
Input point is on the twist! 
Pass
Pass
Pass
Pass
Pass
Input point is on the twist! 
Pass
Pass
Pass
Test case for X25519:
u:
e6db6867583030db3594c1a424b15f7c726624ec26b3353b10a903a6d0ab1c4c
s:
a546e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449ac4
r:
c3da55379de9c6908e94ea4df28d084f32eccf03491c71f754b4075577a28552
Test case for X25519:
u:
e5210f12786811d3f4b7959d0538ae2c31dbe7106fc03c3efc4cd549c715a413
s:
4b66e9d4d1b4673c5ad22691957d6af5c11b6421e0ea01d42ca4169e7918ba0d
r:
95cbde9476e8907d7aade45cb4b873f88b595a68799fa152e6f8f7647aac7957
Test case for X25519:
u:
0000000000000000000000000000000000000000000000000000000000000000
s:
a546e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449ac4
r:
0000000000000000000000000000000000000000000000000000000000000000
Test case for X25519:
u:
0000000000000000000000000000000000000000000000000000000000000000
s:
af46e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449aff

In [6]:
print ("Points that need to return neutral element when input to ")
print ("plain X25519 that also accept un-normalized inputs with")
print ("bit #255 set in the input point encoding.")

ctr=0;
for x in weakp:
    print ("u"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(x));
    ctr += 1;

    
print ("\nResults for X25519 implementations not clearing bit #255:")
print ("s =", IntegerToLEPrintString(s_in));
print ("rN = X25519(uX, s);")
ctr=0;
for x in weakp:
    r = X25519(IntegerToByteArray(s_in), IntegerToByteArray(x),warnForPointOnTwist=0);
    r = ByteArrayToInteger(r)
    print ("r"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(r));
    ctr += 1;


print ("\nResults for X25519 implementations that clear bit #255:")
print ("s =", IntegerToLEPrintString(s_in));
print ("qN = X25519(uX & ((1 << 255) - 1),s);")
ctr=0;
for x in weakp:
    q = X25519(IntegerToByteArray(s_in), IntegerToByteArray(x & ((1<<255)-1)),warnForPointOnTwist=0);
    q = ByteArrayToInteger(q)
    print ("q"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(q));
    ctr += 1;


Points that need to return neutral element when input to 
plain X25519 that also accept un-normalized inputs with
bit #255 set in the input point encoding.
u0: 0000000000000000000000000000000000000000000000000000000000000000
u1: 0100000000000000000000000000000000000000000000000000000000000000
u2: e0eb7a7c3b41b8ae1656e3faf19fc46ada098deb9c32b1fd866205165f49b800
u3: 5f9c95bca3508c24b1d0b1559c83ef5b04445cc4581c8e86d8224eddd09f1157
u4: ecffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u5: edffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u6: eeffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u7: cdeb7a7c3b41b8ae1656e3faf19fc46ada098deb9c32b1fd866205165f49b880
u8: 4c9c95bca3508c24b1d0b1559c83ef5b04445cc4581c8e86d8224eddd09f11d7
u9: d9ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
ua: daffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
ub: dbffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

Results for X25

In [7]:
# 4.) Definitions for the CPace string handling primitives

def prepend_length_to_bytes(data):
    length = len(data);
    length_as_utf8_string = chr(length).encode('utf-8') # Will be a single byte in almost all real-world cases.
    return length_as_utf8_string + data 

def prefix_free_cat(*args):
    result = b""
    for arg in args:
        result += prepend_length_to_bytes(arg)
    return result

def zero_bytes(length):
    result = b"\0" * length
    return result

def generator_string(PRS,DSI,CI,sid,s_in_bytes):
    """
    Concat all input fields with prepended length information.
    Add zero padding in the first hash block between PRS and DSI.
    """
    len_zpad = max(0,s_in_bytes - 1 - len(prepend_length_to_bytes(PRS)))
    return (prefix_free_cat(PRS, zero_bytes(len_zpad), DSI, CI, sid), len_zpad)

def random_bytes(length):
    values = [randint(0, 255) for i in range(length)]
    result = b""
    for v in values:
        result += v.to_bytes(1, 'little')
    return result

def bigger_than(str1,str2):
    if len(str1) > len(str2):
        return True
    if len(str1) < len(str2):
        return False
    for i in range(len(str1)):
        if str1[i] > str2[i]:
            return True
    return False

def oCAT(str1,str2):
    if str1 > str2:
        return str1 + str2
    else:
        return str2 + str1

In [8]:
#Definitions for the hash primitives

import hashlib 

class H_SHA512:
    def __init__(self):
        self.b_in_bytes = 64
        self.bmax_in_bytes = 64
        self.s_in_bytes = 128
        self.name = "SHA512"
        
    def hash(self,input_str, l = 64):
        m = hashlib.sha512(input_str)
        digest = m.digest()
        if len(digest) < l:
            raise ValueError("Output length of Hash primitive (%i bytes) not long enough. %i bytes were requested." % (len(digest), l))
        return digest[0:l]

class H_SHA256:
    def __init__(self):
        self.b_in_bytes = 32
        self.bmax_in_bytes = 32
        self.s_in_bytes = 64
        self.name = "SHA256"
        
    def hash(self,input_str, l = 32):
        m = hashlib.sha256(input_str)
        digest = m.digest()
        if len(digest) < l:
            raise ValueError("Output length of Hash primitive (%i bytes) not long enough. %i bytes were requested." % (len(digest), l))

        return digest[0:l]

class H_SHAKE256:
    def __init__(self):
        self.b_in_bytes = 64
        self.bmax_in_bytes = 2^128
        self.s_in_bytes = 136
        self.name = "SHAKE256"
        
    def hash(self,input_str, l = 64):
        m = hashlib.shake_256(input_str)
        digest = m.digest(l) # Note: hashlib.shake_256 seems to be buggy in some Sage environments :-(
        return digest

In [29]:
class G_Montgomery:
    """Here we have common definitions for the X448 and X25519"""
    def sample_scalar(self, deterministic_scalar_for_test_vectors = "False"):
        if deterministic_scalar_for_test_vectors == "False":
            return random_bytes(self.field_size_bytes)
        else:
            H = H_SHA512()
            value = H.hash(deterministic_scalar_for_test_vectors)
            return value[0:self.field_size_bytes]

    def decodeLittleEndian(self, b):
        bits = self.field_size_bits
        num_bytes = floor((bits+7)/8)
        return sum([b[i] << 8*i for i in range(num_bytes)])

    def decodeUCoordinate(self, u):        
        u_list = [b for b in u]
        # Ignore any unused bits.
        if self.field_size_bits % 8:
            u_list[-1] &= (1<<(self.field_size_bits%8))-1
        return self.decodeLittleEndian(u_list)

    def encodeUCoordinate(self,u):
        u = u % self.q
        return IntegerToByteArray(u,self.field_size_bytes)
    
    def find_z_ell2(self,F):
        """ Argument: F, a field object, e.g., F = GF(2^255 - 19) """
        ctr = F.gen()
        while True:
            for Z_cand in (F(ctr), F(-ctr)):
                # Z must be a non-square in F.
                if is_square(Z_cand):
                    continue
                return Z_cand
            ctr += 1
        
    def elligator2(self,r):
        q = self.q
        Fq = GF(q)
        A = Fq(self.A)
        B = Fq(1)
    
        # calculate the appropriate non-square as specified in the hash2curve draft.
        u = Fq(self.find_z_ell2(Fq))
        powerForChi = floor((q-1)/2)
    
        v = - A / (1 + u * r^2)
        epsilon = (v^3 + A * v^2 + B * v)^powerForChi
        x = epsilon * v - (1 - epsilon) * A/2
        return self.encodeUCoordinate(Integer(x))

    def calculate_generator(self, H, PRS, CI, sid, print_test_vector_info = False):
        (gen_string, len_zpad) = generator_string(PRS, G.DSI,CI,sid,H.s_in_bytes)
        string_hash = H.hash(gen_string, self.field_size_bytes)
        u = self.decodeUCoordinate(string_hash)
        result = self.elligator2(u)
        if print_test_vector_info:
            print ("###  Test vectors for calculate_generator with group",self.name)
            print ("  ~~~")

            print ("  Inputs")
            print ("    H   =", H.name, "with input block size", H.s_in_bytes, "bytes.")
            print ("    PRS =", PRS, "; ZPAD length:", len_zpad,"; DSI =", self.DSI)
            print ("    CI =", CI)
            print ("    CI =", ByteArrayToLEPrintString(CI))
            print ("    sid =", ByteArrayToLEPrintString(sid))
            print ("  Outputs")
            tv_output_byte_array(string_hash, test_vector_name = "hash generator string", 
                                 line_prefix = "    ", max_len = 70)
            tv_output_byte_array(IntegerToByteArray(u), test_vector_name = "after decoding to coordinate", 
                                 line_prefix = "    ", max_len = 70)
            tv_output_byte_array(result, test_vector_name = "generator g", 
                                 line_prefix = "    ", max_len = 70)
            print ("  ~~~")
        return result

    
class G_X25519(G_Montgomery):
    def __init__(self):
        self.I = zero_bytes(32)
        self.field_size_bytes = 32
        self.field_size_bits = 255
        self.DSI = b"CPace255"
        self.DSI_ISK = b"CPace255_ISK"
        self.name = "X25519" # group name
        self.encoding_of_scalar = "little endian"
        
        # curve definitions
        self.q = 2^255 - 19
        self.A = 486662
        
    def scalar_mult(self,scalar,point):
        return X25519(scalar,point)

    def scalar_mult_vfy(self,scalar,point):
        return X25519(scalar,point)

    
class G_X448(G_Montgomery):
    def __init__(self):
        self.I = zero_bytes(56)
        self.field_size_bytes = 56
        self.field_size_bits = 448
        self.DSI = b"CPace448"
        self.DSI_ISK = b"CPace448_ISK"
        self.name = "X448" # group name
        self.encoding_of_scalar = "little endian"
        
        # curve definitions
        self.q = 2^448 - 2^224 - 1
        self.A = 156326
        
    def scalar_mult(self,scalar,point):
        return X448(scalar,point) # yet no definition for X448 in this script file

    def scalar_mult_vfy(self,scalar,point):
        return X448(scalar,point) # yet no definition for X448 in this script file


In [10]:
# 4.) Definitions of the Elligator2 test cases
#
# Elligator 2 test cases
#
#
# Testvector from the NaCl M0 testsuite from E+H
#
G = G_X25519()

etc1_in =  0x00c84eddfa9bcd7973d6021153cd965a8a2fd749135834eaaeb093d2469a14bc
etc1_out = 0x67d305efdb0a7c7f24ce1655ecc103126004ff23d32bfc032428cd75758fb666

etc2_in =  0x7563f23b0c0aa7bc27b2961a4711ba842ba303c57a9534164bf8d3b5d455cf89
etc2_out = 0x08a3bb40e5b594b192d0ee87b663d24e1bc76d2d41c9031962a7ec6cc863b11d

ourResult1 = ByteArrayToInteger(G.elligator2(etc1_in))
ourResult2 = ByteArrayToInteger(G.elligator2(etc2_in))

if (ourResult1 != etc1_out):
    print ("Elligator test case #1 failed.")
else:
    print ("Elligator test case #1 pass.")

if (ourResult2 != etc2_out):
    print ("Elligator test case #2 failed.")
else:
    print ("Elligator test case #2 pass.")
    
print ("Elligator test case #1:")
print ("In:  0x%x" % etc1_in)
print ("Out: 0x%x" % etc1_out)

print ("Elligator test case #1:")
print ("In:  0x%x" % etc2_in)
print ("Out: 0x%x" % etc2_out)


Elligator test case #1 pass.
Elligator test case #2 pass.
Elligator test case #1:
In:  0xc84eddfa9bcd7973d6021153cd965a8a2fd749135834eaaeb093d2469a14bc
Out: 0x67d305efdb0a7c7f24ce1655ecc103126004ff23d32bfc032428cd75758fb666
Elligator test case #1:
In:  0x7563f23b0c0aa7bc27b2961a4711ba842ba303c57a9534164bf8d3b5d455cf89
Out: 0x8a3bb40e5b594b192d0ee87b663d24e1bc76d2d41c9031962a7ec6cc863b11d


In [30]:
def CPace_ISK(DSI,sid,K,MSGa,MSGb,doPrint = 1, symmetric_execution = False):
    if symmetric_execution:
        concatenated_msg_transcript = oCAT(MSGa,MSGb)
        if doPrint:
            print ("###  Test vector for ISK calculation parallel execution")
            print ("~~~")
        tv_output_byte_array(concatenated_msg_transcript, test_vector_name = "ordered cat of transcript ", 
                         line_prefix = "    ", max_len = 60)
    else:
        concatenated_msg_transcript = MSGa + MSGb
        if doPrint:
            print ("###  Test vector for ISK calculation initiator/responder")
            print ("~~~")
        tv_output_byte_array(concatenated_msg_transcript, test_vector_name = "unordered cat of transcript ", 
                         line_prefix = "    ", max_len = 60)
        
    string = prefix_free_cat(DSI,sid,K)+ concatenated_msg_transcript
    m = hashlib.sha512(string)
    ISK = m.digest()
    if doPrint:
        tv_output_byte_array(string, test_vector_name = "input to final ISK hash", 
                         line_prefix = "    ", max_len = 60)
        tv_output_byte_array(ISK, test_vector_name = "ISK result", 
                         line_prefix = "    ", max_len = 60)
        print ("~~~")

    return ISK

In [ ]:
def generate_test_vector(H,G, with_ANSI_C_initializers = True):
    print ("##  Test vector for CPace using group " + G.name + " and hash "+H.name +"\n")

    sid = H.hash(b"sid")
    sid = sid [:16]

    PRS = b"password"
    CI = (prepend_length_to_bytes(b"Ainitiator") 
          + prepend_length_to_bytes(b"Bresponder"))

    ADa = b"ADa"
    ADb = b"ADb"

    g = G.calculate_generator(H,PRS,CI,sid, True)
    
    seed = b""
    while True:
        ya = G.sample_scalar(b"A"+seed)
        Ya = G.scalar_mult(ya, g)
        yb = G.sample_scalar(b"B"+seed)
        Yb = G.scalar_mult(yb, g)
        if not (oCAT(Ya,Yb) == Ya + Yb):
            break;
        seed += b" "
            
    MSGa = prefix_free_cat(Ya,ADa)
    MSGb = prefix_free_cat(Yb,ADb)
   
    print ("###  Test vector for MSGa")
    print ("  ~~~")
    print ("  Inputs")
    print ("    ADa =",ADa)
    tv_output_byte_array(ya, test_vector_name = "ya (" + G.encoding_of_scalar +")", 
                         line_prefix = "    ", max_len = 60)
    print ("  Outputs")
    tv_output_byte_array(Ya, test_vector_name = "Ya", 
                         line_prefix = "    ", max_len = 60)
    tv_output_byte_array(MSGa, test_vector_name = "MSGa", 
                         line_prefix = "    ", max_len = 60)
    print ("  ~~~")
    print ("###  Test vector for MSGb")
    print ("  ~~~")
    print ("  Inputs")
    print ("    ADb =", ADb)
    tv_output_byte_array(yb, test_vector_name = "yb (" + G.encoding_of_scalar +")", 
                         line_prefix = "    ", max_len = 60)
    print ("  Outputs")
    tv_output_byte_array(Yb, test_vector_name = "Yb", 
                         line_prefix = "    ", max_len = 60)
    tv_output_byte_array(MSGb, test_vector_name = "MSGb", 
                         line_prefix = "    ", max_len = 60)
    
    print ("  ~~~")
    print ("###  Test vector for secret points K")
    print ("  ~~~")
    K1 = G.scalar_mult_vfy(ya,Yb)
    K2 = G.scalar_mult_vfy(yb,Ya)
    tv_output_byte_array(K1, test_vector_name = "scalar_mult_vfy(ya,Yb)", 
                         line_prefix = "    ", max_len = 60)
    tv_output_byte_array(K2, test_vector_name = "scalar_mult_vfy(yb,Ya)", 
                         line_prefix = "    ", max_len = 60)
    print ("  ~~~")
    if (K1 != K2):
        print ("Diffie-Hellman did fail!")
    K = K1
    
    ISK_IR = CPace_ISK(G.DSI_ISK,sid,K,MSGa,MSGb,doPrint = 1, symmetric_execution = False)
    ISK_SY = CPace_ISK(G.DSI_ISK,sid,K,MSGa,MSGb,doPrint = 1, symmetric_execution = True)
    
    if with_ANSI_C_initializers:
        print ("###  Corresponding ANSI-C initializers")
        print ("  ~~~")
        print (ByteArrayToCInitializer(PRS, "tc_PRS"))
        print (ByteArrayToCInitializer(CI, "tc_CI"))
        print (ByteArrayToCInitializer(sid, "tc_sid"))
        print (ByteArrayToCInitializer(g, "tc_g"))
        print (ByteArrayToCInitializer(ya, "tc_ya"))
        print (ByteArrayToCInitializer(ADa, "tc_ADa"))
        print (ByteArrayToCInitializer(Ya, "tc_Ya"))
        print (ByteArrayToCInitializer(yb, "tc_yb"))
        print (ByteArrayToCInitializer(ADb, "tc_ADb"))
        print (ByteArrayToCInitializer(Yb, "tc_Yb"))
        print (ByteArrayToCInitializer(K1, "tc_K"))
        print (ByteArrayToCInitializer(ISK_IR, "tc_ISK_IR"))
        print (ByteArrayToCInitializer(ISK_SY, "tc_ISK_SY"))
        print ("  ~~~")


H = H_SHA512()
G = G_X25519()
generate_test_vector(H,G)


In [13]:
H = H_SHAKE256()
G = G_X448()
generate_test_vector(H,G)

##  Test vector for CPace using group X448 and hash SHAKE256

###  Test vectors for calculate_generator with group X448
  Inputs
    H   = SHAKE256 with input block size 136 bytes.
    PRS = b'password' ; ZPAD length: 126 ; DSI = b'CPace448'
    CI = b'\nAinitiator\nBresponder'
    CI = 0a41696e69746961746f720a42726573706f6e646572
    sid = 5223e0cdc45d6575668d64c552004124
  Outputs
    hash generator string: (length: 56 bytes)
      c1658ad06392f4eb5a23294d49210744aea89bf56cd9d1497b0b6ca0d4a9172fed
      d1e9d8376794c166ebbe05b598c051cbad24b03892e84
    after decoding to coordinate: (length: 32 bytes)
      c1658ad06392f4eb5a23294d49210744aea89bf56cd9d1497b0b6ca0d4a9172f
    generator g: (length: 56 bytes)
      402906591ba645f89b94dc93559c9c423a35d5eaf2878da0fd11b912aee50ffbf5
      37a6b3bf72c28f3a12cf521eac520d2630806ee2b2f41
Ya: bytearray(b'\xc7\xb7+\xd1"\xa5\xd1\n\x05h\xc4&\x17\x99\xc9\xb7\xfc\xf9\xa0\x1a\x98\x86\xd8\xdeXK\xe5s\xa4\xb7\xf1*k=\xb9\x11]^Hg\x97\x86g#\x1a\x1b\r:\xaa\

In [14]:
# Test vectors for the C Code, Elligator2

print (ByteArrayToCInitializer(IntegerToByteArray(etc1_in), "EllTestCase1_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc1_out), "EllTestCase1_out"))

print (ByteArrayToCInitializer(IntegerToByteArray(etc2_in), "EllTestCase2_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc2_out), "EllTestCase2_out"))


const uint8_t EllTestCase1_in[] = {
 0xbc,0x14,0x9a,0x46,0xd2,0x93,0xb0,0xae,0xea,0x34,0x58,0x13,0x49,0xd7,0x2f,0x8a,
 0x5a,0x96,0xcd,0x53,0x11,0x02,0xd6,0x73,0x79,0xcd,0x9b,0xfa,0xdd,0x4e,0xc8,0x00,
};
const uint8_t EllTestCase1_out[] = {
 0x66,0xb6,0x8f,0x75,0x75,0xcd,0x28,0x24,0x03,0xfc,0x2b,0xd3,0x23,0xff,0x04,0x60,
 0x12,0x03,0xc1,0xec,0x55,0x16,0xce,0x24,0x7f,0x7c,0x0a,0xdb,0xef,0x05,0xd3,0x67,
};
const uint8_t EllTestCase2_in[] = {
 0x89,0xcf,0x55,0xd4,0xb5,0xd3,0xf8,0x4b,0x16,0x34,0x95,0x7a,0xc5,0x03,0xa3,0x2b,
 0x84,0xba,0x11,0x47,0x1a,0x96,0xb2,0x27,0xbc,0xa7,0x0a,0x0c,0x3b,0xf2,0x63,0x75,
};
const uint8_t EllTestCase2_out[] = {
 0x1d,0xb1,0x63,0xc8,0x6c,0xec,0xa7,0x62,0x19,0x03,0xc9,0x41,0x2d,0x6d,0xc7,0x1b,
 0x4e,0xd2,0x63,0xb6,0x87,0xee,0xd0,0x92,0xb1,0x94,0xb5,0xe5,0x40,0xbb,0xa3,0x08,
};


In [15]:
#

In [16]:
#8.) Test vector output for ANSI-C, CPace




In [17]:
import binascii
import random
import hashlib
import sys

sys.path.insert(0, "./draft-irtf-cfrg-hash-to-curve/poc")
from hash_to_field import *

expander = XMDExpander("DSI", hashlib.sha512, 256)
print(hash_to_field(b"1234", 4, 2^255-19, 1, 64, expander))

expander2 = XOFExpander("DSI", hashlib.shake_256, 256)

print(hash_to_field(b"1234", 4, 2^255-19, 1, 64, expander2))


[[36044779498011844230915766445823177552245649256321920015128326251648804854736], [2807174726284269674247950780443512725521215962584024500047424237459342704766], [3425278505679699565340110266442948439239008931152215885721755692385239661795], [36386770897185740099209420052442887018218493381172789268983985555806060131493]]
[[14770725704879161911398748911644758459473793216159017853048374252821130412052], [50072334603910462271321821409339096908978011876034096131569665899733450902000], [30621651225154301661347584198863852980631618098225193081906096220962083978550], [31402707106732945619889186281313801206550438611251562730578981433940646756415]]


In [18]:
import binascii
import random
import hashlib
import sys

sys.path.insert(0, "./draft-irtf-cfrg-hash-to-curve/poc/sagelib")
from hash_to_field import I2OSP, OS2IP
from suite_p256 import *
from suite_p384 import *
from suite_p521 import *

# Definitions for Short-Weierstrass-Curves

class G_ShortWeierstrass():
    def __init__(self, mapping_primitive):        
        self.map = mapping_primitive
        self.curve = self.map("some arbitrary string").curve();
        
        self.field = self.curve.base_field();
        self.q = self.field.order()
        self.field_size_bytes = ceil(log(float(self.q),2) / 8)
        self.p = self.curve.order();
        self.name = mapping_primitive.curve_name
        
        if not (self.p.is_prime()):
            raise ValueError ("Group order for Short-Weierstrass must be prime")

        self.I = I2OSP(0,1) # Represent the neutral element as a single byte string "\00"

        self.DSI = b"CPace" + mapping_primitive.suite_name.encode("ascii")
        self.DSI_ISK = self.DSI + b"_ISK"
        self.encoding_of_scalar = "big endian"
      
    def sample_scalar(self, deterministic_scalar_for_test_vectors = "False"):
        random_bytes_len =  self.field_size_bytes * 2
        if deterministic_scalar_for_test_vectors == "False":
            string = random_bytes(random_bytes_len)
        else:
            H = H_SHAKE256()
            string = H.hash(deterministic_scalar_for_test_vectors, random_bytes_len)
        scalar = ByteArrayToInteger(string, random_bytes_len)
        scalar = scalar % self.p
        return I2OSP(scalar,self.field_size_bytes)

    def point_to_octets(self,point):
        if point == (point * 0):
            return b"\00" # Neutral element.
    
        x,y = point.xy()
        return b"\04" + I2OSP(x,self.field_size_bytes) + I2OSP(y,self.field_size_bytes)

    def octets_to_point(self,octets):
        if (octets[0] == 0) and (len(octets) == 1):
            point = self.curve.gens()[0]
            return point * 0 # neutral element.
        
        if not octets[0] == 4:
            raise ValueError("Only uncompressed format supported.")
           
        if not (len(octets) == 1 + self.field_size_bytes * 2):
            raise ValueError("Wrong length of field")
        
        xstr = octets[1:(self.field_size_bytes+1)]
        ystr = octets[(self.field_size_bytes+1):]
        return self.curve(OS2IP(xstr),OS2IP(ystr))
        
    def scalar_mult(self,scalar_octets,point):
        scalar = OS2IP(scalar_octets)
        return self.point_to_octets(point * scalar)

    def scalar_mult_vfy(self,scalar_octets,point_octets):
        scalar = OS2IP(scalar_octets)
        try:
            point = self.octets_to_point(point_octets)
        except:
            # Incorrect format or point is not on curve
            return self.I
        
        result_point = point * scalar
               
        if result_point == point * 0:
            return self.I
        else:
            x,y = result_point.xy()
            return I2OSP(x,self.field_size_bytes)
        
    def calculate_generator(self, H, PRS, CI, sid, print_test_vector_info = False):
        (gen_string, len_zpad) = generator_string(PRS, G.DSI,CI,sid,H.s_in_bytes)
        result = self.map(gen_string)
        if print_test_vector_info:
            print ("  ###  Test vectors for calculate_generator with group",self.name," ###")
            print ("  Inputs")
            print ("    H   =", H.name, "with input block size", H.s_in_bytes, "bytes.")
            print ("    PRS =", PRS, "; ZPAD length:", len_zpad,"; DSI =", self.DSI)
            print ("    CI =", CI)
            print ("    CI =", ByteArrayToLEPrintString(CI))
            print ("    sid =", ByteArrayToLEPrintString(sid))
            print ("  Outputs")
            tv_output_byte_array(gen_string, test_vector_name = "string passed to map", 
                                 line_prefix = "    ", max_len = 70)
            tv_output_byte_array(self.point_to_octets(result), test_vector_name = "generator g", 
                                 line_prefix = "    ", max_len = 70)
        return result

prime = 2^255 - 19
F = GF(prime)
A = F(486662)
E = EllipticCurve(F, [0, A , 0, 1 , 0])
point = E.gens()[0]

#print (point_to_octets (point))

#print (point)
#print (octets_to_point(point_to_octets (point),E))

#print (p256_sswu_nu("Hallo.2"))

G_P256 = G_ShortWeierstrass(p256_sswu_nu)

scalar = G_P256.sample_scalar()
#scalar = b"\0"
generator = G_P256.calculate_generator(H_SHA256(),b"PRS", b"ci",b"sid", False)
Y = G_P256.scalar_mult(scalar, generator)

#print (G_P256.octets_to_point(Y))

K = G_P256.scalar_mult_vfy(scalar, Y)

print (scalar)
print (generator)
print (Y)
print (K)

b'\x9b~bP\xfc9\x8c\xa9\x02\xa0\x91\xa4\xac\x83\xc1\xc6\n\xb3"\x16"\x8cG\x81\x066\xf2\xbc\xba\xb2;\xde'
(53015870623411926744193832597175745355940465503795306609868876078855309399495 : 73346833251810743184933851582679510550355903834101479189607145435723045562931 : 1)
b'\x04\xb4P4\xe5\xadI{`\x9e\xc3\x906\xf8R\x13\xd1e\xe6\x98\xa7gO\xe5>\xe3_\xe0\xb0\x94\x07\x90\x94\x93\xf9\xc5N\xe9s?L\xa6\xe4\xa1\x08\x05\x1c\xce*@-\xf0\xfb\xb2\xe6\xcc \x00\xd5\xa3}5s\xae;'
b"%\xcd\xef\xe9\t\xeev,&\xc5\x99]i\xae\xa9\xc5\xb5'\xdc\xac[\xc4C\x068\xa3\x15\xcd\x06Q\x85["


In [19]:
H = H_SHA256()
G = G_ShortWeierstrass(p256_sswu_nu)
#generate_test_vector(H,G)

H = H_SHA512()
G = G_ShortWeierstrass(p384_sswu_nu)
generate_test_vector(H,G)

H = H_SHA512()
G = G_ShortWeierstrass(p521_sswu_nu)
generate_test_vector(H,G)



##  Test vector for CPace using group NIST P-384 and hash SHA512

  ###  Test vectors for calculate_generator with group NIST P-384  ###
  Inputs
    H   = SHA512 with input block size 128 bytes.
    PRS = b'password' ; ZPAD length: 118 ; DSI = b'CPaceP384_XMD:SHA-384_SSWU_NU_'
    CI = b'\nAinitiator\nBresponder'
    CI = 0a41696e69746961746f720a42726573706f6e646572
    sid = 7e4b4791d6a8ef019b936c79fb7f2c57
  Outputs
    string passed to map: (length: 199 bytes)
      0870617373776f7264760000000000000000000000000000000000000000000000
      000000000000000000000000000000000000000000000000000000000000000000
      000000000000000000000000000000000000000000000000000000000000000000
      00000000000000000000000000000000000000000000000000000000001e435061
      6365503338345f584d443a5348412d3338345f535357555f4e555f160a41696e69
      746961746f720a42726573706f6e646572107e4b4791d6a8ef019b936c79fb7
    generator g: (length: 97 bytes)
      047a150362b4d77b37d1af4a74209381886225b869328dbb40ee38

KeyboardInterrupt: 

In [ ]:
import binascii
import random
import hashlib
import sys

sys.path.insert(0, "./draft-irtf-cfrg-voprf/poc/sagelib")
from ristretto_decaf import *

# Definitions for Short-Weierstrass-Curves

class G_CoffeeBaseClass():
    def __init__(self, group_abstraction_point):        
        self.abstraction = group_abstraction
        self.curve = self.map("some arbitrary string").curve();
        
        self.field = self.curve.base_field();
        self.q = self.field.order()
        self.field_size_bytes = ceil(log(q,2) / 8)
        self.p = self.curve.order();
        self.name = mapping_primitive.curve_name
        
        if not (self.p.is_prime()):
            raise ValueError ("Group order for Short-Weierstrass must be prime")

        self.I = I2OSP(0,1) # Represent the neutral element as a single byte string "\00"

        self.DSI = b"CPace" + mapping_primitive.suite_name.encode("ascii")
        self.DSI_ISK = self.DSI + b"_ISK"
        self.encoding_of_scalar = "big endian"
      
    def sample_scalar(self, deterministic_scalar_for_test_vectors = "False"):
        random_bytes_len =  self.field_size_bytes * 2
        if deterministic_scalar_for_test_vectors == "False":
            string = random_bytes(random_bytes_len)
        else:
            H = H_SHAKE256()
            string = H.hash(deterministic_scalar_for_test_vectors, random_bytes_len)
        scalar = ByteArrayToInteger(string, random_bytes_len)
        scalar = scalar % self.p
        return I2OSP(scalar,self.field_size_bytes)

    def point_to_octets(self,point):
        if point == (point * 0):
            return b"\00" # Neutral element.
    
        x,y = point.xy()
        return b"\04" + I2OSP(x,self.field_size_bytes) + I2OSP(y,self.field_size_bytes)

    def octets_to_point(self,octets):
        if (octets[0] == 0) and (len(octets) == 1):
            point = self.curve.gens()[0]
            return point * 0 # neutral element.
        
        if not octets[0] == 4:
            raise ValueError("Only uncompressed format supported.")
           
        if not (len(octets) == 1 + self.field_size_bytes * 2):
            raise ValueError("Wrong length of field")
        
        xstr = octets[1:(self.field_size_bytes+1)]
        ystr = octets[(self.field_size_bytes+1):]
        return self.curve(OS2IP(xstr),OS2IP(ystr))
        
    def scalar_mult(self,scalar_octets,point):
        scalar = OS2IP(scalar_octets)
        return point_to_octets(point * scalar)

    def scalar_mult_vfy(self,scalar_octets,point_octets):
        scalar = OS2IP(scalar_octets)
        try:
            point = self.octets_to_point(point_octets)
        except:
            # Incorrect format or point is not on curve
            return self.I
        
        result_point = point * scalar
               
        if result_point == point * 0:
            return self.I
        else:
            x,y = result_point.xy()
            return I2OSP(x,self.field_size_bytes)
        
    def calculate_generator(self, H, PRS, CI, sid, print_test_vector_info = False):
        (gen_string, len_zpad) = generator_string(PRS, G.DSI,CI,sid,H.s_in_bytes)
        result = self.map(gen_string)
        if print_test_vector_info:
            print ("  ###  Test vectors for calculate_generator with group",self.name," ###")
            print ("  Inputs")
            print ("    H   =", H.name, "with input block size", H.s_in_bytes, "bytes.")
            print ("    PRS =", PRS, "; ZPAD length:", len_zpad,"; DSI =", self.DSI)
            print ("    CI =", CI)
            print ("    CI =", ByteArrayToLEPrintString(CI))
            print ("    sid =", ByteArrayToLEPrintString(sid))
            print ("  Outputs")
            tv_output_byte_array(gen_string, test_vector_name = "string passed to map", 
                                 line_prefix = "    ", max_len = 70)
            tv_output_byte_array(point_to_octets(result), test_vector_name = "generator g", 
                                 line_prefix = "    ", max_len = 70)
        return result


class G_CoffeeEcosystem():
    def __init__(self, coffee_point_class):
        self.point_class = coffee_point_class
        self.field = coffee_point_class.F
        self.q = coffee_point_class.P
        self.field_size_bytes = ceil(log(self.q,2) / 8)
        self.p = coffee_point_class.order
        self.name = coffee_point_class.name
        
        self.I = (coffee_point_class.map(H_SHAKE256().hash(b"1234",self.field_size_bytes * 2)) * 0).encode()
        self.DSI = b"CPace" + self.name.encode("ascii")
        self.DSI_ISK = self.DSI + b"_ISK"
        self.encoding_of_scalar = "little endian"

    def sample_scalar(self, deterministic_scalar_for_test_vectors = "False"):
        if deterministic_scalar_for_test_vectors == "False":
            value = random_bytes(self.field_size_bytes)
        else:
            value = H_SHAKE256().hash(deterministic_scalar_for_test_vectors)
        
        value_int = ByteArrayToInteger(value, self.field_size_bytes)
        reduced_value = value_int % self.p
        result = IntegerToByteArray(reduced_value,self.field_size_bytes)        
        return result

    def scalar_mult(self,scalar,point):
        scalar_as_int = ByteArrayToInteger(scalar, self.field_size_bytes);
        return (point * scalar_as_int).encode()

    def scalar_mult_vfy(self,scalar,encoded_point):
        scalar_as_int = ByteArrayToInteger(scalar, self.field_size_bytes);
        try:
            point = self.point_class.decode(encoded_point);
        except:
            # Decoding of point failed.
            return self.I
        return (point * scalar_as_int).encode()

    def calculate_generator(self, H, PRS, CI, sid, print_test_vector_info = False):
        (gen_string, len_zpad) = generator_string(PRS, G.DSI,CI,sid,H.s_in_bytes)
        string_hash = H.hash(gen_string, self.field_size_bytes * 2)
        result = self.point_class.map(string_hash)
        if print_test_vector_info:
            print ("  ###  Test vectors for calculate_generator with group",self.name," ###")
            print ("  Inputs")
            print ("    H   =", H.name, "with input block size", H.s_in_bytes, "bytes.")
            print ("    PRS =", PRS, "; ZPAD length:", len_zpad,"; DSI =", self.DSI)
            print ("    CI =", CI)
            print ("    CI =", ByteArrayToLEPrintString(CI))
            print ("    sid =", ByteArrayToLEPrintString(sid))
            print ("  Outputs")
            tv_output_byte_array(gen_string, test_vector_name = "hash generator string", 
                                 line_prefix = "    ", max_len = 70)
            tv_output_byte_array(string_hash, test_vector_name = "hash result", 
                                 line_prefix = "    ", max_len = 70)
            tv_output_byte_array(result.encode(), test_vector_name = "encoded generator g", 
                                 line_prefix = "    ", max_len = 70)
        return result

H = H_SHA512()
G = G_CoffeeEcosystem(Ed25519Point)

H = H_SHAKE256()
G = G_CoffeeEcosystem(Ed448GoldilocksPoint)
generate_test_vector(H,G)


P255 = Ed25519Point()

P448 = Ed448GoldilocksPoint()

(P255.map(H_SHA512().hash(b"1234")) * 3).encode()

G.I

In [ ]:

print("Hallo", file = sys.stdout)


###  Test vector for MSGa
  ~~~
  Inputs
    ADa = b'ADa'
    ya (little endian): (length: 32 bytes)
      232527dee2cfde76fb425b6d88818630eea7ea263fac28d89f52d096
      c563b1e
  Outputs
    Ya: (length: 32 bytes)
      5448fd9633734e703210b61d5cabb1310a28382895d56d490551436a
      b339864
    MSGa: (length: 37 bytes)
      205448fd9633734e703210b61d5cabb1310a28382895d56d49055143
      6ab33986440341446
  ~~~
###  Test vector for MSGb
  ~~~
  Inputs
    ADb = b'ADb'
    yb (little endian): (length: 32 bytes)
      871ebb1d5ecbeffa5a47e32c40d2da6894d9f2865efdad6ad8535a1b
      e7e487d
  Outputs
    Yb: (length: 32 bytes)
      d8fe025158c0c08d7ea93a84718a56111bff54bf4b960c8343e64f02
      5eead60
    MSGb: (length: 37 bytes)
      20d8fe025158c0c08d7ea93a84718a56111bff54bf4b960c8343e64f
      025eead6080341446
  ~~~
###  Test vector for secret points K
  ~~~
    scalar_mult_vfy(ya,Yb): (length: 32 bytes)
      4aa59ccfda03691c3e9cf4dab329a13bcc9707e38f54e784e30f7843
      78dbcb4
    scalar_mult_vfy(yb,Ya): (length: 32 bytes)
      4aa59ccfda03691c3e9cf4dab329a13bcc9707e38f54e784e30f7843
      78dbcb4
###  Test vector for ISK calculation initiator/responder
    unordered cat of transcript : (length: 74 bytes)
      205448fd9633734e703210b61d5cabb1310a28382895d56d49055143
      6ab33986440341446120d8fe025158c0c08d7ea93a84718a56111bff
      54bf4b960c8343e64f025eead608034144
    input to final ISK hash: (length: 137 bytes)
      0c43506163653235355f49534b107e4b4791d6a8ef019b936c79fb7f
      2c57204aa59ccfda03691c3e9cf4dab329a13bcc9707e38f54e784e3
      0f784378dbcb49205448fd9633734e703210b61d5cabb1310a283828
      95d56d490551436ab33986440341446120d8fe025158c0c08d7ea93a
      84718a56111bff54bf4b960c8343e64f025eead6080341
    ISK result: (length: 64 bytes)
      eeaebbac8a5a057cf94c3e8dba6cf0edddb953357532d0028e3e0cd4
      a85ce7a4b42511d5fb06c60f3b0775c357f267ccc9e24b81338231b9
      a61855fcebd4a0
###  Test vector for ISK calculation parallel execution
    ordered cat of transcript : (length: 74 bytes)
      20d8fe025158c0c08d7ea93a84718a56111bff54bf4b960c8343e64f
      025eead60803414462205448fd9633734e703210b61d5cabb1310a28
      382895d56d490551436ab3398644034144
    input to final ISK hash: (length: 137 bytes)
      0c43506163653235355f49534b107e4b4791d6a8ef019b936c79fb7f
      2c57204aa59ccfda03691c3e9cf4dab329a13bcc9707e38f54e784e3
      0f784378dbcb4920d8fe025158c0c08d7ea93a84718a56111bff54bf
      4b960c8343e64f025eead60803414462205448fd9633734e703210b6
      1d5cabb1310a28382895d56d490551436ab33986440341
    ISK result: (length: 64 bytes)
      170a1df42f03da082b9d250be646be04c8c6b4490af17b27c4a0916f
      e8482289d10ea0534455ab578deb88dae75db4a8bf24fb111825e630
      2ad3c0d903341f
  ~~~
###  Corresponding ANSI-C initializers
  ~~~
const uint8_t tc_PRS[] = {
 0x70,0x61,0x73,0x73,0x77,0x6f,0x72,0x64,
};
const uint8_t tc_CI[] = {
 0x0a,0x41,0x69,0x6e,0x69,0x74,0x69,0x61,0x74,0x6f,0x72,0x0a,0x42,0x72,0x65,0x73,
 0x70,0x6f,0x6e,0x64,0x65,0x72,
};
const uint8_t tc_sid[] = {
 0x7e,0x4b,0x47,0x91,0xd6,0xa8,0xef,0x01,0x9b,0x93,0x6c,0x79,0xfb,0x7f,0x2c,0x57,
};
const uint8_t tc_g[] = {
 0x2c,0xdd,0xcc,0x94,0xb3,0x8d,0x05,0x9a,0x7b,0x30,0x5b,0xb0,0xb8,0x93,0x4b,0x5b,
 0x1e,0xd4,0x5c,0x5a,0x5c,0xb0,0x39,0xf9,0xcd,0x00,0xab,0x11,0xce,0x92,0x73,0x0d,
};
const uint8_t tc_ya[] = {
 0x23,0x25,0x27,0xde,0xe2,0xcf,0xde,0x76,0xfb,0x42,0x5b,0x6d,0x88,0x81,0x86,0x30,
 0xee,0xa7,0xea,0x26,0x3f,0xac,0x28,0xd8,0x9f,0x52,0xd0,0x96,0xc5,0x63,0xb1,0xe6,
};
const uint8_t tc_ADa[] = {
 0x41,0x44,0x61,
};
const uint8_t tc_Ya[] = {
 0x54,0x48,0xfd,0x96,0x33,0x73,0x4e,0x70,0x32,0x10,0xb6,0x1d,0x5c,0xab,0xb1,0x31,
 0x0a,0x28,0x38,0x28,0x95,0xd5,0x6d,0x49,0x05,0x51,0x43,0x6a,0xb3,0x39,0x86,0x44,
};
const uint8_t tc_yb[] = {
 0x87,0x1e,0xbb,0x1d,0x5e,0xcb,0xef,0xfa,0x5a,0x47,0xe3,0x2c,0x40,0xd2,0xda,0x68,
 0x94,0xd9,0xf2,0x86,0x5e,0xfd,0xad,0x6a,0xd8,0x53,0x5a,0x1b,0xe7,0xe4,0x87,0xd6,
};
const uint8_t tc_ADb[] = {
 0x41,0x44,0x62,
};
const uint8_t tc_Yb[] = {
 0xd8,0xfe,0x02,0x51,0x58,0xc0,0xc0,0x8d,0x7e,0xa9,0x3a,0x84,0x71,0x8a,0x56,0x11,
 0x1b,0xff,0x54,0xbf,0x4b,0x96,0x0c,0x83,0x43,0xe6,0x4f,0x02,0x5e,0xea,0xd6,0x08,
};
const uint8_t tc_K[] = {
 0x4a,0xa5,0x9c,0xcf,0xda,0x03,0x69,0x1c,0x3e,0x9c,0xf4,0xda,0xb3,0x29,0xa1,0x3b,
 0xcc,0x97,0x07,0xe3,0x8f,0x54,0xe7,0x84,0xe3,0x0f,0x78,0x43,0x78,0xdb,0xcb,0x49,
};
const uint8_t tc_ISK_IR[] = {
 0xee,0xae,0xbb,0xac,0x8a,0x5a,0x05,0x7c,0xf9,0x4c,0x3e,0x8d,0xba,0x6c,0xf0,0xed,
 0xdd,0xb9,0x53,0x35,0x75,0x32,0xd0,0x02,0x8e,0x3e,0x0c,0xd4,0xa8,0x5c,0xe7,0xa4,
 0xb4,0x25,0x11,0xd5,0xfb,0x06,0xc6,0x0f,0x3b,0x07,0x75,0xc3,0x57,0xf2,0x67,0xcc,
 0xc9,0xe2,0x4b,0x81,0x33,0x82,0x31,0xb9,0xa6,0x18,0x55,0xfc,0xeb,0xd4,0xa0,0x26,
};
const uint8_t tc_ISK_SY[] = {
 0x17,0x0a,0x1d,0xf4,0x2f,0x03,0xda,0x08,0x2b,0x9d,0x25,0x0b,0xe6,0x46,0xbe,0x04,
 0xc8,0xc6,0xb4,0x49,0x0a,0xf1,0x7b,0x27,0xc4,0xa0,0x91,0x6f,0xe8,0x48,0x22,0x89,
 0xd1,0x0e,0xa0,0x53,0x44,0x55,0xab,0x57,0x8d,0xeb,0x88,0xda,0xe7,0x5d,0xb4,0xa8,
 0xbf,0x24,0xfb,0x11,0x18,0x25,0xe6,0x30,0x2a,0xd3,0xc0,0xd9,0x03,0x34,0x1f,0x83,
};
  ~~~